In [78]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

highlevel_dirs = sum([
    #["../tools"], 
    #sorted(glob.glob("../sem16*")),
    sorted(glob.glob("../sem17*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem17-sockets-tcp']


In [79]:
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        

../sem17-sockets-tcp ['../sem17-sockets-tcp/sockets-tcp.ipynb']
[NbConvertApp] Converting notebook ../sem17-sockets-tcp/sockets-tcp.ipynb to markdown
[NbConvertApp] Writing 45525 bytes to ../sem17-sockets-tcp/README.md


In [80]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [81]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./../sem17-sockets-tcp/sockets-tcp.ipynb to Unix format ...
dos2unix: converting file ./../sem17-sockets-tcp/README.md to Unix format ...


In [83]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem17-sockets-tcp/*.ipynb
fatal: pathspec '../sem17-sockets-tcp/*.ipynb' did not match any files
> git add --ignore-errors  ../sem17-sockets-tcp/*.md
fatal: pathspec '../sem17-sockets-tcp/*.md' did not match any files
> git add --ignore-errors  ../sem17-sockets-tcp/*.c
fatal: pathspec '../sem17-sockets-tcp/*.c' did not match any files
> git add --ignore-errors  ../sem17-sockets-tcp/*.cpp
fatal: pathspec '../sem17-sockets-tcp/*.cpp' did not match any files
> git add --ignore-errors -f  -f ../sem17-sockets-tcp/bash_popen_tmp/*.html
fatal: pathspec '../sem17-sockets-tcp/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem17-sockets-tcp/interactive_launcher_tmp/*.log
fatal: pathspec '../sem17-sockets-tcp/interactive_launcher_tmp/*.log' did not match any files
> git add -u
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
> git commit 